In [1]:
! pip install -U langchain-community sentence-transformers huggingface_hub accelerate transformers faiss-cpu datasets ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.2/484.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [2]:
from transformers import pipeline
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain.llms import HuggingFacePipeline
from sentence_transformers import SentenceTransformer
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
import os
from getpass import getpass

from datasets import Dataset
from ragas import evaluate, RunConfig
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)


from huggingface_hub import login
import pandas as pd

login()

In [3]:
# https://huggingface.co/datasets/virattt/financial-qa-10K/viewer?row=0&views%5B%5D=train
df = pd.read_parquet("hf://datasets/virattt/financial-qa-10K/data/train-00000-of-00001.parquet")
df = df.loc[df.ticker == "NVDA"]
df = df[["question", "answer", "context"]]
print(df.shape)
df.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(100, 3)


,question,answer,context
0,What area did NVIDIA initially focus on before...,NVIDIA initially focused on PC graphics.,"Since our original focus on PC graphics, we ha..."
1,What are some of the recent applications of GP...,Recent applications of GPU-powered deep learni...,Some of the most recent applications of GPU-po...
2,What significant invention did NVIDIA create i...,NVIDIA invented the GPU in 1999.,Our invention of the GPU in 1999 defined moder...
3,How does NVIDIA's platform strategy contribute...,NVIDIA's platform strategy brings together har...,"NVIDIA has a platform strategy, bringing toget..."
4,What does NVIDIA's CUDA programming model enable?,NVIDIA's CUDA programming model opened the par...,With our introduction of the CUDA programming ...


In [4]:
class CustomEmbeddings(Embeddings):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, normalize_embeddings=True).tolist()

    def embed_query(self, query):
        return self.embed_documents([query])[0]

# Sample documents
docs = df.context.apply(lambda x: {"page_content": x}).to_list()

# Create vector store
embedding_model = CustomEmbeddings()
texts = [d["page_content"] for d in docs]
doc_embeddings = embedding_model.embed_documents(texts)
vector_store = FAISS.from_texts(texts, embedding_model)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
llm_pipeline = pipeline("text2text-generation", model="meta-llama/Llama-3.2-1B-Instruct", model_kwargs={"temperature": 0.2})
llm = HuggingFacePipeline(pipeline=llm_pipeline)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu
The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'

In [6]:
# Define a plain string prompt for text2text generation
prompt = PromptTemplate.from_template(
    "Answer the question using only the context provided.\n"
    "If the answer is not in the context, say 'I don't know'.\n\n"
    "Context:\n{context}\n\nQuestion:\n{input}\n\nAnswer:"
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [7]:
chain = create_retrieval_chain(vector_store.as_retriever(), question_answer_chain)
query = "What are some of the recent applications of GPU-powered deep learning as mentioned by NVIDIA?"
answer = chain.invoke({"input": query})
print("Respuesta:", answer["answer"].split("Answer:")[1])
print("Contexto:", answer["context"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Respuesta:  
- Recommendation systems
- Large language models
- Generative AI
- Deep learning and machine
Contexto: [Document(id='5648830f-64b7-462b-9ec8-4c8e2e5c82c8', metadata={}, page_content='Some of the most recent applications of GPU-powered deep learning include recommendation systems, which are AI algorithms trained to understand the preferences, previous decisions, and characteristics of people and products using data gathered about their interactions, large language models, which can recognize, summarize, translate, predict and generate text and other content based on knowledge gained from massive datasets, and generative AI, which uses algorithms that create new content, including audio, code, images, text, simulations, and videos, based on the data they have been trained on.'), Document(id='21e69f60-bade-44a9-b2f3-2666f6c0abab', metadata={}, page_content='We provide a complete, end-to-end accelerated computing platform for deep learning and machine learning, addressing both

# LangChain chains

In [10]:
import os
from getpass import getpass

# Use getpass to securely prompt for the API key
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [11]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

new_chain = (
    {"context": vector_store.as_retriever(),  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(new_chain.invoke(query))

Recommendation systems, large language models, and generative AI.


# RAGAS

In [12]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(vector_store.as_retriever(), question_answer_chain)
retriever = vector_store.as_retriever()


samples = []
gt_contexts = []
retrieved_context_lists = []
for i, row in df.iterrows():
    question = row["question"]
    gt_answer = row["answer"]
    gt_context = row["context"]
    result = chain.invoke({"input": question})

    retrieved_docs = retriever.get_relevant_documents(question)
    contexts = [doc.page_content for doc in retrieved_docs]

    # Save for evaluation
    gt_contexts.append(gt_context.strip())
    retrieved_context_lists.append(contexts)


    samples.append({
        "question": question,
        "answer": result["answer"] if isinstance(result, dict) else result,
        "contexts": contexts,
        "ground_truth": gt_answer,
    })

    if i >= 2:  # Evaluate on a subset for speed
        break

# Convert to HuggingFace Dataset
dataset = Dataset.from_list(samples)

# Evaluate with RAGAS
ragas_results = evaluate(dataset, metrics=[faithfulness, context_precision, answer_relevancy])
print(ragas_results)

<ipython-input-12-08166afdebab>:15: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]

{'faithfulness': 1.0000, 'context_precision': 0.8056, 'answer_relevancy': 0.8243}


In [13]:
def has_overlap(gt, retrieved):
    return any(gt in ctx for ctx in retrieved)

retriever_hits = [has_overlap(gt, retrieved) for gt, retrieved in zip(gt_contexts, retrieved_context_lists)]
retriever_recall_at_k = sum(retriever_hits) / len(retriever_hits)

print(f"Retriever Recall@k: {retriever_recall_at_k:.2%}")

Retriever Recall@k: 100.00%


# Resources

In [ ]:
# https://www.youtube.com/watch?v=sVcwVQRHIc8
# https://arxiv.org/pdf/2409.13731
# https://arxiv.org/html/2412.15605v1
# https://arxiv.org/pdf/2410.05779
# https://arxiv.org/pdf/2502.14902
# https://arxiv.org/pdf/2312.10997